In [7]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema


In [38]:
# Initialize parameters
window = 100
gain_window = 5
extractor = ALPClientExtractor(os.getenv("APCAKEY"), os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
tickers = [x + "/USD" for x in tickers]

prices = []
start = datetime.now() - timedelta(days=1)
end = datetime.now()

# Loop through each ticker to fetch prices and calculate metrics
for ticker in tickers:
    response = extractor.crypto_interval(ticker, start)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c": "adjclose", "t": "date"})[["date", "adjclose"]]
    
    # Convert date and sort
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    
    # Calculate metrics
    price["rolling_return"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"]) / price["adjclose"])
    price["bollinger"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"].rolling(window).std() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(gain_window)
    price["avg_return"] = price["return"].pct_change(gain_window).rolling(window).mean()
    price["coev"] = (price["adjclose"].rolling(window).mean() / price["adjclose"].rolling(window).std())
    price["ticker"] = ticker
    prices.append(price)

# Combine all prices into a single DataFrame
sim = pd.concat(prices).sort_values("date").dropna()

In [39]:
metrics = [
    "rolling_return",
    "bollinger",
    "return",
    "avg_return",
    "coev"
]

In [40]:
import copy
analysis = []
date_range = list(sim["date"].unique())
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
for metric in metrics:
    for ascending in [True,False]:
        cash = 100
        position = copy.deepcopy(base_position)
        cashes = []
        for date in tqdm(date_range):
            try:
                position = copy.deepcopy(position)
                position["date"] = date
                today = sim[sim["date"]==date]
                if today.index.size >= 0:
                    opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        pnl_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] >= 0.005)
                        loss_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] <= -0.005)
                        hpr_req = (date - position["buy_date"]).days > float(5/(24*60))
                        if hpr_req and position["ticker"] != opportunity["ticker"]:
                            try:
                                cash = position["adjclose"] * position["quantity"] * 0.99
                                cashes.append(cash)
                                position = copy.deepcopy(base_position)
                                position["date"] = date
                                position["ticker"] = opportunity["ticker"]
                                position["quantity"] = cash / opportunity["adjclose"] 
                                position["adjclose"] = opportunity["adjclose"]
                                position["buy_price"] = opportunity["adjclose"]
                                position["buy_date"] = date
                            except Exception as e:
                                continue
                    else:
                        position["date"] = date
                        position["ticker"] = opportunity["ticker"]
                        position["quantity"] = cash / opportunity["adjclose"] 
                        position["adjclose"] = opportunity["adjclose"]
                        position["buy_price"] = opportunity["adjclose"]
                        position["buy_date"] = date
            except Exception as e:
                continue
        position["metric"] = metric
        position["ascending"] = ascending
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(copy.deepcopy(position))

100%|████████████████████████████████████████████████████████████████████████████████████████| 2290/2290 [00:09<00:00, 239.67it/s]


In [41]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,pv
1,2024-09-27 16:03:00+00:00,CRV/USD,347.422750,0.314895,0.298620,2024-09-27 01:58:00+00:00,rolling_return,False,109.401687
3,2024-09-27 16:03:00+00:00,CRV/USD,347.087474,0.314895,0.298086,2024-09-27 01:53:00+00:00,bollinger,False,109.296110
4,2024-09-27 16:03:00+00:00,YFI/USD,0.019525,5444.620000,5304.477000,2024-09-27 01:52:00+00:00,return,True,106.307142
9,2024-09-27 16:03:00+00:00,UNI/USD,14.111466,7.514500,7.339480,2024-09-27 01:52:00+00:00,coev,False,106.040615
0,2024-09-27 16:03:00+00:00,LTC/USD,1.509152,70.180500,68.628500,2024-09-27 01:52:00+00:00,rolling_return,True,105.913030
5,2024-09-27 16:03:00+00:00,LTC/USD,1.509152,70.180500,68.628500,2024-09-27 01:52:00+00:00,return,False,105.913030
7,2024-09-27 16:03:00+00:00,LTC/USD,1.509152,70.180500,68.628500,2024-09-27 01:52:00+00:00,avg_return,False,105.913030
2,2024-09-27 16:03:00+00:00,XTZ/USD,139.809053,0.753315,0.740802,2024-09-27 01:52:00+00:00,bollinger,True,105.320257
6,2024-09-27 16:03:00+00:00,BAT/USD,536.961920,0.196065,0.192883,2024-09-27 01:52:00+00:00,avg_return,True,105.279439
8,2024-09-27 16:03:00+00:00,BAT/USD,536.961920,0.196065,0.192883,2024-09-27 01:52:00+00:00,coev,True,105.279439
